In [1]:
import petlink32
import numpy as np
import time

In [2]:
#prints from Siemens walkthrough, a bed position tag
ew1 =  0xc40d180c
hbp = 0xd180c
hbp_se = -190452

In [3]:
my_hbp = petlink32.HORIZONTAL_BED_POSITION.evaluate(hbp)
print("hbp diff = {}".format(hbp-my_hbp))

hbp diff = 0


In [4]:
my_hbp_se = petlink32.sign_extend(my_hbp, 20)
print(my_hbp_se)
diff = hbp_se - my_hbp_se
print("hbp_se diff = {}".format(diff))

-190452
hbp_se diff = 0


In [5]:
print(petlink32.sign_extend(np.array([my_hbp]*3), 20))

[-190452 -190452 -190452]


In [6]:
# packet construction from PETLink document
doc_string = ["PET Bin-Address Event Packet", 
              "0PBB BBBB BBBB BBBB BBBB BBBB BBBB BBBB",
              "P: Prompt (1 - Prompt; 0 - Delay)",
              "B: Bin Address: 0-29"] 
packet_name = doc_string[0]
part_names = ["0: Tag bit "] + doc_string[2:]
print(part_names)
packet_instruction = doc_string[1]
packet = petlink32.packet(packet_instruction, packet_name, part_names)
print(packet)
tag = 1 << 31
event = 23 
print("packet compare: event = {}, tag = {}".format(packet.compare(event), packet.compare(tag)))
print("event packet evaluation ", packet.evaluate(event))

['0: Tag bit ', 'P: Prompt (1 - Prompt; 0 - Delay)', 'B: Bin Address: 0-29']
packet(name = PET Bin-Address Event Packet
instruction = 0PBB BBBB BBBB BBBB BBBB BBBB BBBB BBBB
parts = 
0: Tag bit 
(lowbit, highbit, nbits) = (31, 31, 1)
value = 0 (base 2)
mask =  1000 0000 0000 0000 0000 0000 0000 0000
---
P: Prompt (1 - Prompt; 0 - Delay)
(lowbit, highbit, nbits) = (30, 30, 1)
mask =  0100 0000 0000 0000 0000 0000 0000 0000
---
B: Bin Address: 0-29
(lowbit, highbit, nbits) = (0, 29, 30)
mask =  0011 1111 1111 1111 1111 1111 1111 1111
)
packet compare: event = True, tag = False
event packet evaluation  [0, 0, 23]


In [7]:
# the field names are often nicely ordered, a document parser seems difficult however
# "comparable" parts, with preset values, e.g. 010,  are not named
doc_string = "Position Profile Data (Maintenance)"
doc_string += "\n0uuu uHHH BBBB BBBh XXXX XXXX YYYY YYYY "
doc_string += "\n0: Tag bit" # my addition so that each part has a name
doc_string += "\nu: Undefined (Default: Set to zero.)"
doc_string += "\nH: Head number: 0-2 [See 3 H-field bits labelled by \"210 \" above.]"
doc_string += "\nB: Block position: 0-6"
doc_string += "\nh: Hit bit (always = 1)"
doc_string += "\nX: X position data: 0-7"
doc_string += "\nY: Y position data: 0-7"

In [8]:
print(doc_string)
doc_strings = doc_string.split("\n")
print(doc_strings)
packet_name = doc_strings[0]
packet_instruction = doc_strings[1]
packet_part_names = doc_strings[2:]

Position Profile Data (Maintenance)
0uuu uHHH BBBB BBBh XXXX XXXX YYYY YYYY 
0: Tag bit
u: Undefined (Default: Set to zero.)
H: Head number: 0-2 [See 3 H-field bits labelled by "210 " above.]
B: Block position: 0-6
h: Hit bit (always = 1)
X: X position data: 0-7
Y: Y position data: 0-7
['Position Profile Data (Maintenance)', '0uuu uHHH BBBB BBBh XXXX XXXX YYYY YYYY ', '0: Tag bit', 'u: Undefined (Default: Set to zero.)', 'H: Head number: 0-2 [See 3 H-field bits labelled by "210 " above.]', 'B: Block position: 0-6', 'h: Hit bit (always = 1)', 'X: X position data: 0-7', 'Y: Y position data: 0-7']


In [9]:
complicated_packet = petlink32.packet(packet_instruction, packet_name, packet_part_names)
print(complicated_packet)

packet(name = Position Profile Data (Maintenance)
instruction = 0uuu uHHH BBBB BBBh XXXX XXXX YYYY YYYY 
parts = 
0: Tag bit
(lowbit, highbit, nbits) = (31, 31, 1)
value = 0 (base 2)
mask =  1000 0000 0000 0000 0000 0000 0000 0000
---
u: Undefined (Default: Set to zero.)
(lowbit, highbit, nbits) = (27, 30, 4)
mask =  0111 1000 0000 0000 0000 0000 0000 0000
---
H: Head number: 0-2 [See 3 H-field bits labelled by "210 " above.]
(lowbit, highbit, nbits) = (24, 26, 3)
mask =  0000 0111 0000 0000 0000 0000 0000 0000
---
B: Block position: 0-6
(lowbit, highbit, nbits) = (17, 23, 7)
mask =  0000 0000 1111 1110 0000 0000 0000 0000
---
h: Hit bit (always = 1)
(lowbit, highbit, nbits) = (16, 16, 1)
mask =  0000 0000 0000 0001 0000 0000 0000 0000
---
X: X position data: 0-7
(lowbit, highbit, nbits) = (8, 15, 8)
mask =  0000 0000 0000 0000 1111 1111 0000 0000
---
Y: Y position data: 0-7
(lowbit, highbit, nbits) = (0, 7, 8)
mask =  0000 0000 0000 0000 0000 0000 1111 1111
)


In [10]:
complicated_packet = petlink32.packet(doc_string)
print(complicated_packet)

packet(name = Position Profile Data (Maintenance)
instruction = 0uuu uHHH BBBB BBBh XXXX XXXX YYYY YYYY 
parts = 
0: Tag bit
(lowbit, highbit, nbits) = (31, 31, 1)
value = 0 (base 2)
mask =  1000 0000 0000 0000 0000 0000 0000 0000
---
u: Undefined (Default: Set to zero.)
(lowbit, highbit, nbits) = (27, 30, 4)
mask =  0111 1000 0000 0000 0000 0000 0000 0000
---
H: Head number: 0-2 [See 3 H-field bits labelled by "210 " above.]
(lowbit, highbit, nbits) = (24, 26, 3)
mask =  0000 0111 0000 0000 0000 0000 0000 0000
---
B: Block position: 0-6
(lowbit, highbit, nbits) = (17, 23, 7)
mask =  0000 0000 1111 1110 0000 0000 0000 0000
---
h: Hit bit (always = 1)
(lowbit, highbit, nbits) = (16, 16, 1)
mask =  0000 0000 0000 0001 0000 0000 0000 0000
---
X: X position data: 0-7
(lowbit, highbit, nbits) = (8, 15, 8)
mask =  0000 0000 0000 0000 1111 1111 0000 0000
---
Y: Y position data: 0-7
(lowbit, highbit, nbits) = (0, 7, 8)
mask =  0000 0000 0000 0000 0000 0000 1111 1111
)


In [11]:
doc_string = "Horizontal Bed Position"
doc_string += "\n1100 0100 uuuM HHHH HHHH HHHH HHHH HHHH"
#|19 0| # not use
doc_string += "\nu: Undefined (Default: Set to zero.)"

doc_string += "\nM: Moving. Single Bit. Set to 0 if horizontal bed motion is unchanging. " #moved up for correct order
doc_string += "Set to 1 if horizontal bed motion is changing. "
doc_string += "[This bit was requested but implementation may be pending.]"

doc_string += "\nH: Horizontal Position: 0-19 [Note: For VG30 on Nov-2010 and later mCT systems, "
doc_string += "this 20-bit H value is always a 2’s complement number. "
doc_string += "When H is near zero - e.g. H = -1(dec) or FFFFF(hex), "
doc_string += "the bed (a. k. a. Patient Handling System or PHS) "
doc_string += "approaches (or is) fully retracted from the FOV. "
doc_string += "When H is at its largest negative extent, "
doc_string += "the bed is fully extended into the FOV. "
doc_string += "The LSB of H represents 0.001 cm of horizontal bed movement. "
doc_string += "In recent years, the actual precision reported by " 
doc_string += "the bed has been 0.001cm – i.e. to the LSB limit of the H field. " 
doc_string += "Some earlier versions of the PHS had precision limited to 0.05cm " 
doc_string += "– i.e. incrementing/decrementing the H field by 50 (dec) only.]" 

In [12]:
print(doc_string)

Horizontal Bed Position
1100 0100 uuuM HHHH HHHH HHHH HHHH HHHH
u: Undefined (Default: Set to zero.)
M: Moving. Single Bit. Set to 0 if horizontal bed motion is unchanging. Set to 1 if horizontal bed motion is changing. [This bit was requested but implementation may be pending.]
H: Horizontal Position: 0-19 [Note: For VG30 on Nov-2010 and later mCT systems, this 20-bit H value is always a 2’s complement number. When H is near zero - e.g. H = -1(dec) or FFFFF(hex), the bed (a. k. a. Patient Handling System or PHS) approaches (or is) fully retracted from the FOV. When H is at its largest negative extent, the bed is fully extended into the FOV. The LSB of H represents 0.001 cm of horizontal bed movement. In recent years, the actual precision reported by the bed has been 0.001cm – i.e. to the LSB limit of the H field. Some earlier versions of the PHS had precision limited to 0.05cm – i.e. incrementing/decrementing the H field by 50 (dec) only.]


In [13]:
hbp_packet = petlink32.packet(doc_string, is_twos_complement = [False, False, False, True])

In [14]:
print(hbp_packet)

packet(name = Horizontal Bed Position
instruction = 1100 0100 uuuM HHHH HHHH HHHH HHHH HHHH
parts = 
u: Undefined (Default: Set to zero.)
(lowbit, highbit, nbits) = (24, 31, 8)
value = 11000100 (base 2)
mask =  1111 1111 0000 0000 0000 0000 0000 0000
---
M: Moving. Single Bit. Set to 0 if horizontal bed motion is unchanging. Set to 1 if horizontal bed motion is changing. [This bit was requested but implementation may be pending.]
(lowbit, highbit, nbits) = (21, 23, 3)
mask =  0000 0000 1110 0000 0000 0000 0000 0000
---
H: Horizontal Position: 0-19 [Note: For VG30 on Nov-2010 and later mCT systems, this 20-bit H value is always a 2’s complement number. When H is near zero - e.g. H = -1(dec) or FFFFF(hex), the bed (a. k. a. Patient Handling System or PHS) approaches (or is) fully retracted from the FOV. When H is at its largest negative extent, the bed is fully extended into the FOV. The LSB of H represents 0.001 cm of horizontal bed movement. In recent years, the actual precision report

In [15]:
print(hbp_packet.evaluate(0xc40d180c))

[196, 0, 0, -190452]


In [21]:
doc_group = petlink32.document_section(name = "my section", packets = [packet, hbp_packet])
print(doc_group)
doc_group.pages = "lorem ipsum"
print("\nwith pages:\n")
print(doc_group)

section(name = my section
, packets = 
PET Bin-Address Event Packet
Horizontal Bed Position )

with pages:

section(name = my section
, packets = 
PET Bin-Address Event Packet
Horizontal Bed Position
, pages = lorem ipsum )
